<img src='https://www.unifor.br/o/unifor-theme/images/unifor-logo-horizontal.svg' width="250px">

# DATA HARVESTING / Projeto da Disciplina

Prof.: Ms. Alex Lima<br>
MBA em Ciência de Dados<br>
Universidade de Fortaleza

**Gabriela Ferreira Coutinho - 2418581**</br>

# 1. Capturando as URLs para Web Scraping

## 1.1 Instalação e Configuração 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# 🔹 Configuração do Selenium para evitar bloqueios
chrome_options = Options()
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36")

# 🔹 Inicializa o WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

print("✅ Selenium configurado com sucesso!")

## 1.2 Acessar a Página Principal

In [ ]:
# 🔹 URL da página principal do torneio
url_base = "https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7"
driver.get(url_base)

# 🔹 Espera a página carregar totalmente
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
time.sleep(5)  # Tempo extra para garantir carregamento

print("✅ Página carregada com sucesso!")

## 1.3 Abrir o Dropdown e Coletar as Temporadas

In [ ]:
# 🔹 Clicar no botão do dropdown para abrir a lista de temporadas
try:
    dropdown_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CLASS_NAME, "DropdownButton"))
    )
    dropdown_button.click()
    time.sleep(3)  # Espera os itens carregarem
except Exception as e:
    print("🚨 Erro ao abrir o dropdown das temporadas:", e)
    driver.quit()
    exit()

# 🔹 Coletar os IDs das temporadas
temporadas_urls = {}

try:
    # 🔹 Buscar a lista de temporadas toda vez antes de interagir
    temporadas_elements = driver.find_elements(By.XPATH, "//ul[@role='listbox']/li")
    temporadas_textos = [item.text.strip() for item in temporadas_elements]

    for nome_temporada in temporadas_textos:
        # 🔹 Reabre o dropdown para garantir que ele ainda está disponível
        dropdown_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "DropdownButton"))
        )
        dropdown_button.click()
        time.sleep(2)

        # 🔹 Rebusca a lista de temporadas antes de clicar
        temporadas_elements = driver.find_elements(By.XPATH, "//ul[@role='listbox']/li")

        for item in temporadas_elements:
            if item.text.strip() == nome_temporada:
                item.click()  # Seleciona a temporada para carregar a página correta
                time.sleep(3)

                # 🔹 Coletar o ID da URL após a seleção
                temporada_id = driver.current_url.split("#id:")[-1]
                url_final = f"https://www.sofascore.com/pt/torneio/futebol/europe/uefa-champions-league/7#id:{temporada_id}"
                temporadas_urls[nome_temporada] = url_final
                break

except Exception as e:
    print("🚨 Erro ao coletar as temporadas:", e)

print("✅ Temporadas coletadas com sucesso!")


## 1.4 Mostrar as 10 Temporadas Coletadas

In [ ]:
# 🔹 Filtrar apenas as 10 últimas temporadas
temporadas_filtradas = dict(list(temporadas_urls.items())[:10])

# 🔹 Exibe os IDs e URLs corrigidas
print("\n📌 Temporadas encontradas no Sofascore:")
for temporada, url in temporadas_filtradas.items():
    print(f"{temporada}: {url}")


## 1.5 Fechar o navegador 

In [ ]:
# 🔹 Fechar o navegador ao fim do processo
driver.quit()
print("✅ Navegador fechado com sucesso!")

## 1.6 Salvar temporadas como CSV

In [ ]:
import os
import pandas as pd

# 🔹 Criar a pasta 'data' caso não exista
os.makedirs("data", exist_ok=True)

# 🔹 Caminho do arquivo CSV
csv_path = "data/temporadas_urls.csv"

# 🔹 Converter dicionário para DataFrame e salvar
df_temporadas = pd.DataFrame(list(temporadas_filtradas.items()), columns=["Temporada", "URL"])
df_temporadas.to_csv(csv_path, index=False, encoding="utf-8")

print(f"✅ Temporadas salvas em {csv_path}!")


# 2. Capturar dados dos jogadores das temporadas

## 2.1 Carregar as Temporadas do CSV

In [ ]:
import pandas as pd

# 🔹 Caminho do arquivo CSV salvo anteriormente
csv_path = "data/temporadas_urls.csv"

# 🔹 Carregar o CSV e transformar em dicionário {Temporada: URL}
df_temporadas = pd.read_csv(csv_path)
temporadas_urls = dict(zip(df_temporadas["Temporada"], df_temporadas["URL"]))

print(f"✅ {len(temporadas_urls)} temporadas carregadas do CSV para scraping.")


## 2.2 Função para Coletar Dados de uma Temporada

### 2.2.1 Configuração Inicial

In [ ]:
# 🔹 Importação das bibliotecas necessárias
import os
import time
import random
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# 🔹 Configuração do Selenium para evitar bloqueios
chrome_options = Options()
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36")

# 🔹  
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# 🔹 Criar a pasta "data" para armazenar os resultados
os.makedirs("data", exist_ok=True)

print("✅ Configuração inicial concluída!")


### 2.2.2 Carregar Temporadas do CSV

In [ ]:
# 🔹 Carregar as URLs das temporadas do arquivo CSV
csv_path = "data/temporadas_urls.csv"

if os.path.exists(csv_path):
    df_temporadas = pd.read_csv(csv_path)
    temporadas_urls = dict(zip(df_temporadas["Temporada"], df_temporadas["URL"]))
    print(f"✅ {len(temporadas_urls)} temporadas carregadas do CSV para scraping.")
else:
    print("❌ Arquivo de temporadas não encontrado. Execute a etapa de coleta de URLs primeiro!")

### 2.2.3 Função para Coletar Estatísticas dos Jogadores

In [ ]:
# 🔹 Mapeamento das abas desejadas para coleta de estatísticas
abas_desejadas = {
    "attack": "Ataque",
    "defence": "Defesa",
    "passing": "Passe",
    "goalkeeper": "Goleiro"
}

def coletar_dados_temporada(temporada, url):
    print(f"\n🔄 Acessando temporada {temporada}: {url}")

    # Abrindo a página com Selenium
    driver.get(url)
    time.sleep(5)

    try:
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.TAG_NAME, "table"))
        )

        for aba_id, nome_aba in abas_desejadas.items():
            print(f"\n🟡 Coletando dados da aba {nome_aba}...")

            # Clicar na aba correta
            try:
                aba_botao = driver.find_element(By.XPATH, f"//button[@data-tabid='{aba_id}']")
                driver.execute_script("arguments[0].click();", aba_botao)
                time.sleep(3)  # Tempo para a página carregar os novos dados
            except:
                print(f"❌ Não foi possível acessar a aba {nome_aba}. Pulando...")
                continue

            # Capturar a tabela da aba
            try:
                tabela = driver.find_element(By.TAG_NAME, "table")

                # Capturar cabeçalhos
                cabecalhos = [th.text.strip() for th in tabela.find_elements(By.TAG_NAME, "th")]
                cabecalhos.insert(1, "Time")  # Adicionar a coluna "Time"

                dados_totais = []

                # Capturar linhas da tabela
                linhas = tabela.find_elements(By.TAG_NAME, "tr")
                for linha in linhas[1:]:
                    colunas = linha.find_elements(By.TAG_NAME, "td")

                    if colunas:
                        try:
                            # Pega o nome do time na imagem
                            time_element = colunas[1].find_element(By.TAG_NAME, "img")
                            nome_time = time_element.get_attribute("alt") if time_element else "Desconhecido"
                        except:
                            nome_time = "Desconhecido"

                        # Captura os valores das colunas
                        dados_linha = [coluna.text.strip() for coluna in colunas]

                        # Capturar "Nota Sofascore" corretamente
                        try:
                            nota_element = colunas[-1].find_element(By.XPATH, ".//span")
                            nota_sofascore = nota_element.text.strip() if nota_element else ""
                        except:
                            nota_sofascore = ""

                        # Adicionar a nota ao final da linha
                        dados_linha.append(nota_sofascore)

                        # Inserir o nome do time na segunda posição
                        dados_linha.insert(1, nome_time)

                        dados_totais.append(dados_linha)

                # Criar a pasta específica para a aba
                pasta_aba = f"data/{nome_aba}"
                os.makedirs(pasta_aba, exist_ok=True)

                # Salvar em CSV
                if dados_totais:
                    df = pd.DataFrame(dados_totais, columns=cabecalhos + ["Nota Sofascore"])
                    nome_arquivo = f"{pasta_aba}/estatisticas_jogadores_{temporada.replace('/', '-')}.csv"
                    df.to_csv(nome_arquivo, index=False, encoding="utf-8")
                    print(f"✅ Dados da aba {nome_aba} salvos em {nome_arquivo}")
                else:
                    print(f"❌ Nenhum dado encontrado na aba {nome_aba}.")

            except Exception as e:
                print(f"⚠️ Erro ao coletar dados da aba {nome_aba}: {e}")

    except Exception as e:
        print(f"🚨 Erro ao carregar a página da temporada {temporada}: {e}")

### 2.2.4 Executar Web Scraping para Todas as Temporadas

In [ ]:
# 🔹 Executar a coleta de estatísticas para todas as temporadas
for temporada, url in temporadas_urls.items():
    coletar_dados_temporada(temporada, url)

# 🔹 Fechar o navegador ao final do processo
driver.quit()
print("✅ Web scraping concluído! Navegador fechado.")

KeyboardInterrupt: 